<a href="https://colab.research.google.com/github/kryptobolt07/Paytm/blob/main/Ml%20programs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time


nltk.download('treebank')
o
nltk.download('universal_tagset')

nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))


print(nltk_data[:2])

for sent in nltk_data[:2]:
    for tuple in sent:
        print(tuple)


train_set, test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)


train_tagged_words = [tup for sent in train_set for tup in sent ]
test_tagged_words = [tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

# check some of the tagged words.
print(train_tagged_words[:5])

# use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list) #total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]

    count_w_given_tag = len(w_given_tag_list)

    return (count_w_given_tag, count_tag)

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags_list = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags_list if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags_list)-1):
        if tags_list[index] == t1 and tags_list[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)


tags_list = list(tags)
tags_matrix = np.zeros((len(tags_list), len(tags_list)), dtype='float32')
for i, t1 in enumerate(tags_list):
    for j, t2 in enumerate(tags_list):
        count_t2_t1, count_t1 = t2_given_t1(t2, t1)
        tags_matrix[i, j] = count_t2_t1 / count_t1 if count_t1 != 0 else 0

print(tags_matrix)


tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

print(tags_df)

def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            count_w_given_tag, count_tag = word_given_tag(word, tag)
            # Added check for division by zero
            emission_p = count_w_given_tag / count_tag if count_tag != 0 else 0

            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        state_max = T[p.index(pmax)]
        state.append(state_max)

    return list(zip(words, state))


random.seed(1234)


rnom = [random.randint(0,len(test_set) - 1) for x in range(10)] #


test_run = [test_set[i] for i in rnom]

test_run_base = [tup for sent in test_run for tup in sent]

test_tagged_words = [tup[0] for sent in test_run for tup in sent]


start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)


check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]

accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def jaccard_similarity(a, b):
    a_set, b_set = set(a.lower().split()), set(b.lower().split())
    return len(a_set & b_set) / len(a_set | b_set)

def tfidf_cosine_similarity(a, b):
    vec = TfidfVectorizer()
    tfidf = vec.fit_transform([a, b])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

def word2vec_similarity(a, b):
    a_tok, b_tok = word_tokenize(a.lower()), word_tokenize(b.lower())
    model = Word2Vec([a_tok, b_tok], vector_size=100, window=3, min_count=1, workers=1)
    wa = sum(model.wv[w] for w in a_tok) / len(a_tok)
    wb = sum(model.wv[w] for w in b_tok) / len(b_tok)
    return cosine_similarity([wa], [wb])[0][0]

a = "My laptop screams louder than me during exams."
b = "My laptop cries every time I open Chrome."

print("\nJaccard Similarity:", jaccard_similarity(a, b))
print("TF-IDF Cosine Similarity:", tfidf_cosine_similarity(a, b))
print("Word2Vec Similarity:", word2vec_similarity(a, b))

In [ ]:
import pandas as pd
import re
import nltk
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams

text = """
The phone is fast.
The phone is smooth.
The phone is powerful.
The phone is reliable.
"""

# Tokenization
tokens = [w.lower() for w in re.findall(r"\b\w+\b", text)]

# Bigram Model
N = 2
train_data, vocab = padded_everygram_pipeline(N, [tokens])

lm_bigram = MLE(N)
lm_bigram.fit(train_data, vocab)

print("\nN=2 (Bigram Model)")
bigram_counts = {}
for token in set(tokens):
    bigram_counts[token] = dict(lm_bigram.counts[(token,)])
print("All bigram counts:", bigram_counts)

# Prediction - 3 words
context = 'the'
predicted_words = [context]
for i in range(3):
    if lm_bigram.counts[(context,)]:
        next_word = max(lm_bigram.counts[(context,)].items(), key=lambda x: x[1])[0]
        predicted_words.append(next_word)
        context = next_word
print(f"Bigram prediction: {' '.join(predicted_words)}")

# Trigram Model
N = 3
train_data, vocab = padded_everygram_pipeline(N, [tokens])

lm_trigram = MLE(N)
lm_trigram.fit(train_data, vocab)

print("\nN=3 (Trigram Model)")
trigram_counts = {}
bigrams_in_text = list(ngrams(tokens, 2))
for bg in set(bigrams_in_text):
    trigram_counts[bg] = dict(lm_trigram.counts[bg])
print("All trigram counts:", trigram_counts)

# Prediction - 3 words
context = ['the']
predicted_words = context.copy()
for i in range(3):
    if lm_trigram.counts[tuple(context)]:
        next_word = max(lm_trigram.counts[tuple(context)].items(), key=lambda x: x[1])[0]
        predicted_words.append(next_word)
        context = [context[0], next_word]
print(f"Trigram prediction: {' '.join(predicted_words)}")